<div align="center"> <b> <span style='color:red'> <h1> IS 362 - PROJECT 4 </h1> </span> </b> </div>
<div align="center"> <b> <span style='color:#006600;'> <h2> PREDICTIVE ANALYSIS USING SCI-KIT </h2> </span> </b> </div>
<div align="center"> <b> <span style='color:#006600;'> <h4> <span style='color: red;'> Author: </span>Rickardo Henry </h4> </span> </b> </div>

<h4 style='color: red;'><span style='color: #006600;'>Class:</span> IS-362</h4>
<h1> Predictive Analysis & Sci-Kit Learn </h1>


<b> Assignment Tasks:</b>
<ol type='1'>

<li> Start with the mushroom [data](https://archive.ics.uci.edu/ml/datasets/Mushroom) in the pandas DataFrame that you constructed in your <i>“Assignment – Preprocessing Data with Sci-Kit learn.”</i></li>

<li> Use Sci-Kit learn to determine which of the two predictor columns that you selected <i>(odor and one other column of your choice)</i> most accurately predicts whether or not a mushroom is poisonous. There is an additional challenge here — to use Sci-Kit-learn’s predictive classifiers, you’ll want to convert each of your two <i>(numeric categorical)</i> predictor columns into a set of columns. See for one approach pandas get_dummies() method.</li>

<li> Clearly state your conclusions along with any recommendations for further analysis. </li>
</ol>


<b> We begin by importing the necessary modules. </b>

In [86]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression

import sklearn.model_selection


<b> We begin by reading the data. </b>

In [87]:
mushroom_data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data',
                            sep = ',',
                            header = None,
                            usecols=[0, 5, 22],
                            names = ['Mushroom_Classes', 'Odor', 'Habitat']
                           )

mushroom_data.head(10)

,Mushroom_Classes,Odor,Habitat
0,p,p,u
1,e,a,g
2,e,l,m
3,p,p,u
4,e,n,g
5,e,a,g
6,e,a,m
7,e,l,m
8,p,p,g
9,e,a,m


<b> Next, we want to to convert the 'Odor' and 'Habitat' columns into set of columns. 
For this approach, we will use the pandas get_dummies() method. 

We begin by creating the dataframe. </b> (I used the format from [Chris Albon](https://chrisalbon.com/python/pandas_convert_categorical_to_dummies.html)) 

In [88]:
raw_data = {'Mush_Types': ['p', 'e', 'p', 'e', 'p', 'e', 'p', 'e', 'p'], 
            'Odors': ['a', 'l', 'c', 'y', 'f', 'm', 'n', 'p', 's'], 
            'Habitats': ['g', 'l', 'm', 'p', 'u', 'w', 'd', 'g', 'l']
           }

df = pd.DataFrame(raw_data, columns = ['Mush_Types', 'Odors', 'Habitats'])
df.head()

,Mush_Types,Odors,Habitats
0,p,a,g
1,e,l,l
2,p,c,m
3,e,y,p
4,p,f,u


<b> Here we give numerical values to Odors and Habitat using the get_dummies() method. </b>

In [89]:
o_dummy = pd.get_dummies(df['Odors'])
o_dummy

,a,c,f,l,m,n,p,s,y
0,1,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,0,0,1,0


In [90]:
h_dummy = pd.get_dummies(df['Habitats'])
h_dummy

,d,g,l,m,p,u,w
0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0
5,0,0,0,0,0,0,1
6,1,0,0,0,0,0,0
7,0,1,0,0,0,0,0
8,0,0,1,0,0,0,0


<b> Now we proceed to join the columns to the main dataframe. </b>

In [91]:
df_new = pd.concat([df, o_dummy, h_dummy], axis = 1)
df_new

,Mush_Types,Odors,Habitats,a,c,f,l,m,n,p,s,y,d,g,l,m,p,u,w
0,p,a,g,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,e,l,l,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,p,c,m,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,e,y,p,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4,p,f,u,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
5,e,m,w,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
6,p,n,d,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
7,e,p,g,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
8,p,s,l,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0


<b> Now that we have joined the dataframe, we can proceed to determine which of the two predictor columns
chosen most accurately predicts whether or not a mushroom is poisonous.

We define X and y variables appropriately. </b>

In [92]:
X = o_dummy.iloc[:, :-1].values
y = h_dummy.iloc[:, 1].values
X,y

(array([[1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1]], dtype=uint8),
 array([1, 0, 0, 0, 0, 0, 0, 1, 0], dtype=uint8))

<b> Now we setup the training and testing models. </b>

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(6, 8)
(6,)
(3, 8)
(3,)


<b> We continue by using linear regression to predict y value with a test 
variable & use sci-kit learn to predict true & predictive output. </b>

In [94]:
linreg = sklearn.linear_model.LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)
t = [1, 0]
p = [1, 0]


print(sklearn.metrics.mean_absolute_error(t, p))
print(sklearn.metrics.mean_squared_error(t, p))
print(np.sqrt(sklearn.metrics.mean_squared_error(t, p)))

0.0
0.0
0.0


<b> Now we calculate the root mean squared error for the data to figure out margin of error. Shows very little error margin. </b>

In [95]:
X = o_dummy.iloc[:, 2:10].values
y = h_dummy.iloc[:, 1].values
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)

print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

1.0


<b> Remove odor to determine whether or not it can predict edibility. </b>

In [96]:
X = o_dummy.iloc[:, 2:10].values
y = h_dummy.iloc[:, 1].values

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)
linreg.fit(X_train, y_train)
Y_pred = linreg.predict(X_test)

print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

1.0


<b> Conclusion: Based on the sqaure root and low margin error the habitat and odor are not dependent to determine if the mushroom is in fact poisonous or not. </b>


<div align="center"> <b> <span style='color:#006600;'> <h1> THANK YOU! </h1> </span> </b> </div>